In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
import glob
import io
import os
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
pd.options.display.float_format = '{:.2f}'.format

In [2]:
a_flight = pd.read_csv('input/flights/DL89_228f611c.csv')

In [3]:
a_flight.head()

,Timestamp,UTC,Callsign,Position,Altitude,Speed,Direction
0,1571684146,2019-10-21T18:55:46Z,DAL89,"33.945415,-118.410347",0,0,264
1,1571688008,2019-10-21T20:00:08Z,DAL89,"33.945431,-118.410126",0,9,264
2,1571688026,2019-10-21T20:00:26Z,DAL89,"33.945442,-118.409904",0,9,258
3,1571688044,2019-10-21T20:00:44Z,DAL89,"33.945438,-118.40966",0,9,250
4,1571688065,2019-10-21T20:01:05Z,DAL89,"33.945442,-118.409454",0,2,236


In [4]:
a_flight.dtypes

Timestamp     int64
UTC          object
Callsign     object
Position     object
Altitude      int64
Speed         int64
Direction     int64
dtype: object

In [5]:
path = 'input/flights/'

In [6]:
files = glob.glob(os.path.join(path, "*.csv"))

In [7]:
file_df = (pd.read_csv(f, encoding = "ISO-8859-1", low_memory=False)\
           .assign(flightid=os.path.basename(f)) for f in files)

In [8]:
concatenated_df = pd.concat(file_df, ignore_index=True)

In [9]:
concatenated_df['flightid'] = concatenated_df['flightid'].str.replace('DL89_','').str.replace('.csv','')

In [10]:
len(concatenated_df)

196276

In [11]:
concatenated_df.columns = concatenated_df.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [12]:
flights = pd.DataFrame(concatenated_df.sort_values(by='timestamp', ascending=False))

In [13]:
flights.head()

,timestamp,utc,callsign,position,altitude,speed,direction,flightid
100664,1579032628,2020-01-14T20:10:28Z,DAL89,"33.947285,-118.404953",0,12,132,23856f50
100663,1579032616,2020-01-14T20:10:16Z,DAL89,"33.947453,-118.405182",0,15,132,23856f50
100662,1579032605,2020-01-14T20:10:05Z,DAL89,"33.947697,-118.405479",0,19,118,23856f50
100661,1579032598,2020-01-14T20:09:58Z,DAL89,"33.947823,-118.405685",0,20,101,23856f50
100660,1579032592,2020-01-14T20:09:52Z,DAL89,"33.947857,-118.405907",0,21,84,23856f50


In [14]:
flights.tail()

,timestamp,utc,callsign,position,altitude,speed,direction,flightid
12752,1547498470,2019-01-14T20:41:10Z,DAL89,"33.948875,-118.403038",0,20,2,1f2e4d5f
12751,1547498460,2019-01-14T20:41:00Z,DAL89,"33.948624,-118.402939",0,17,36,1f2e4d5f
12750,1547498450,2019-01-14T20:40:50Z,DAL89,"33.94836,-118.403061",0,17,64,1f2e4d5f
12749,1547498441,2019-01-14T20:40:41Z,DAL89,"33.948231,-118.403275",0,13,78,1f2e4d5f
12748,1547498313,2019-01-14T20:38:33Z,DAL89,"33.94812,-118.403481",0,0,87,1f2e4d5f


In [15]:
# !mkdir output

In [16]:
flights.to_csv('output/flights.csv')